# This script fetches all of the relevant data from the EO Run 7 test plan and formats it for insertion to the confluence database
### Requirements
- The [`ts_planning_tool`](https://github.com/lsst-ts/ts_planning_tool/tree/develop) package, which allows you to use the Zephyr Scale API
- A text file named `credentials.txt` that contains the following three lines at the top of the file:
    - ZEPHYR_TOKEN: A token for the Zephyr Scale API
    - JIRA_API_TOKEN: A token for the JIRA api
    - JIRA_USERNAME: Your jira username, which is usually your email (ex. seanmacb@umich.edu)
- Due to sub-shells, you also have to define these environment variables directly within the `cli.py` file

## Setup

#### Imports

In [1]:
import sys
import os
sys.path.append("/Users/sean/Desktop/Repos/ts_planning_tool/")
sys.path.append("/Users/sean/Desktop/Repos/ts_planning_tool/python/lsst/ts/planning/tool/")
import cli as zcli
import zephyr_interface as zint
import tests
import numpy as np
import pandas as pd
import time
import requests
from requests.auth import HTTPBasicAuth
import warnings
# Settings the warnings to be ignored - this is prevalent with the numpydatetime64 object
warnings.filterwarnings('ignore') 
base_web_link = "https://s3df.slac.stanford.edu/data/rubin/lsstcam"
base_weekly = "w_2024_35"

#### Setting appropriate environment variables

In [2]:
export_vars = ["ZEPHYR_TOKEN","JIRA_API_TOKEN","JIRA_USERNAME"]

with open('credentials.txt','r') as f:
    for variables in export_vars:
        line = f.readline()[:-1]
        os.environ[variables] = line

#### Setting headers for an HTTP authentication
- These headers are used for querying the usernames of whoever executed the test case

In [3]:
headers = {'Content-Type': 'application/json','Accept': 'application/json'}
auth = HTTPBasicAuth("seanmacb@umich.edu", os.environ["JIRA_API_TOKEN"])
zepint = zint.ZephyrInterface(jira_api_token=os.environ["JIRA_API_TOKEN"],jira_username=os.environ["JIRA_USERNAME"])
zepint.zephyr_api_token = os.environ["ZEPHYR_TOKEN"]

#### Naming all of the relevant test cases
- Our test cases for Run 7 were `BLOCK-R66 - BLOCK-R78` and `BLOCK-R95`. If there are other test cases you would like to add for querying, this block is where you should do that 

In [4]:
test_cases_arr = []
for k in range(13):
    test_cases_arr.append("BLOCK-R{}".format(66+k))
test_cases_arr.append("BLOCK-R95")

#### A helper function to format database entries

In [5]:
def formatter(execu):
    e_num = execu['key'][6:]
    date = np.datetime64(execu['actualEndDate'])
    # splitTime = date.split("T")
    # base_str = ""
    # for entry in splitTime:
    #     base_str += entry+" "
    # base_str = base_str[:-1]
    # date = np.datetime64(base_str)
    # shifter = zepint.get_user_name(execu["execuutedById"]) # Not yet implemented, working on it
    step_config_parse = zepint.parse(execu["testCase"])

    name_url = zepint.jira_base_url+"user?accountId="+execu["executedById"]
    name_response = requests.get(name_url,headers=headers,auth=auth)
    shifter = name_response.json()["displayName"]
    
    step_config = (step_config_parse)
    E2V_seq = execu["customFields"]["E2V sequencer config"]
    ITL_seq = execu["customFields"]["ITL sequencer config"]
    Corner_seq = execu["customFields"]["Corner sequencer config"]
    CCS_distrib = execu["customFields"]["CCS Distribution"] # Not yet implemented
    comments = execu["comment"]
    hv = execu["customFields"]["HV on?"]
    status = (zepint.parse(execu["testExecutionStatus"]))
    weekly_dist = base_weekly if execu["customFields"]["Pipeline weekly distribution"]==None else execu["customFields"]["Pipeline weekly distribution"]

    if type(comments)!=type(None):
        comments = comments.replace("<br>",", ")
    
    return {"Run #":e_num,"Date":date,"Shifter":shifter,
            "step/config":step_config,"E2V Sequencer file":E2V_seq,
            "ITL Sequencer File":ITL_seq,"Corner Sequencer File":Corner_seq,
            "CCS Distribution":CCS_distrib,"HV on?":hv,"Zephyr Comments":comments,"Status":status,
            "Web report":"{}/{}/{}/".format(base_web_link,e_num,weekly_dist),"User comments":""}

## A cutoff date to only query data from after a certain date
- If you would like to query all data, enter `2024-07-01`

In [6]:
cutoff_date = np.datetime64("2024-09-14")

## The kernel that queries the data, formats it into a dataframe, and writes the dataframe to a file named `ZephyrRecords.csv`

In [7]:
first = False
for block in test_cases_arr:
    a = await zepint.list_test_executions(block,max_results=10000)
    if len(a['values'])!=0:
        for execution in a['values']:
            if (await (zepint.parse(execution["testExecutionStatus"])))['name'] != "Not Executed" and np.datetime64(execution['actualEndDate'].split("T")[0])>cutoff_date:
                result = formatter(execution)
                result["Status"] = (await result["Status"])['name']
                # result["Shifter"] = await result["Shifter"]
                result["step/config"] = (await result["step/config"])["name"].split(" ")[-1] 
                if not first:
                    myDF = pd.DataFrame(columns=list(result.keys()))
                    first=True
                myDF.loc[len(myDF.index)] =list(result.values())
myDF.sort_values("Date",ascending=False).to_csv("{}/ZephyrRecords.csv".format(os.getcwd()),header=True,index=False)
print("Finished, .csv contains {} entries".format(len(myDF)))

Finished, .csv contains 7 entries
